# Module Imports

In [23]:
import pandas as pd
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.decomposition import PCA
from sklearn.preprocessing import PolynomialFeatures
import warnings
import numpy as np
warnings.filterwarnings('ignore')
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from math import sqrt

# Data Formatting

In [24]:
read_file = pd.read_excel ("10_02_2023_Allison Padron.xlsx")
read_file.to_csv ("10_02_2023_Allison Padron.csv", index = None, header=True)

natalie_all = pd.read_csv('10_02_2023_Allison Padron.csv')
natalie_all.drop_duplicates()
natalie_all = natalie_all[:-1]


mikala_numeric = natalie_all.drop(columns=['Date','Opponent','Unnamed: 1'])
mikala_numeric["Chances, %"] = np.where(mikala_numeric["Chances, %"].str.endswith('%'), mikala_numeric["Chances, %"].str.slice(stop=-1), '0')
mikala_numeric["Challenges won, %"] = np.where(mikala_numeric["Challenges won, %"].str.endswith('%'), mikala_numeric["Challenges won, %"].str.slice(stop=-1), '0')
mikala_numeric["Challenges in attack won, %"] = np.where(mikala_numeric["Challenges in attack won, %"].str.endswith('%'), mikala_numeric["Challenges in attack won, %"].str.slice(stop=-1), '0')
mikala_numeric["Challenges in defence won, %"] = np.where(mikala_numeric["Challenges in defence won, %"].str.endswith('%'), mikala_numeric["Challenges in defence won, %"].str.slice(stop=-1), '0')
mikala_numeric["Air challenges won, %"] = np.where(mikala_numeric["Air challenges won, %"].str.endswith('%'), mikala_numeric["Air challenges won, %"].str.slice(stop=-1), '0')
mikala_numeric["Successful dribbles, %"] = np.where(mikala_numeric["Successful dribbles, %"].str.endswith('%'), mikala_numeric["Successful dribbles, %"].str.slice(stop=-1), '0')
mikala_numeric["Tackles won, %"] = np.where(mikala_numeric["Tackles won, %"].str.endswith('%'), mikala_numeric["Tackles won, %"].str.slice(stop=-1), '0')

mikala_numeric = mikala_numeric.mask((mikala_numeric == '-'), 0)
mikala_numeric["Score"] = mikala_numeric["Score"].mask(mikala_numeric["Score"].str[0] > mikala_numeric["Score"].str[-1], 100)
mikala_numeric["Score"] = mikala_numeric["Score"].mask(mikala_numeric["Score"].str[0] == mikala_numeric["Score"].str[-1], 0)
mikala_numeric["Score"] = mikala_numeric["Score"].mask(mikala_numeric["Score"].str[0] < mikala_numeric["Score"].str[-1], -100)


mikala_numeric = mikala_numeric.apply(pd.to_numeric, errors='ignore')


mikala_numeric


,Score,InStat Index,Position,Goals,Assists,Chances,Chances successful,"Chances, %",Сhances created,Fouls,...,Attacking challenges,"Challenges in attack won, %",Air challenges,"Air challenges won, %",Dribbles,"Successful dribbles, %",Tackles,"Tackles won, %",Ball interceptions,Free ball pick ups
0,100,113,CD,0,0,0,0,0,0,1,...,2,50,1,0,0,0,2,0,4,3
1,100,117,DM,0,0,0,0,0,0,0,...,4,25,1,0,0,0,4,50,4,5
2,100,177,DM,0,1,0,0,0,1,1,...,7,43,1,0,5,60,5,80,8,9
3,0,139,CD,0,0,0,0,0,0,2,...,2,100,1,0,0,0,4,75,8,10
4,0,130,CD,0,0,0,0,0,0,0,...,1,100,5,20,0,0,2,50,16,3
5,100,139,CD,0,0,0,0,0,0,0,...,2,100,4,50,0,0,9,56,13,9
6,-100,124,RD,0,0,0,0,0,0,1,...,5,60,0,0,1,100,4,50,3,10
7,-100,125,DM,0,0,0,0,0,0,0,...,2,50,0,0,0,0,0,0,0,1
8,100,129,CM,0,0,0,0,0,0,0,...,1,100,0,0,0,0,1,0,1,0
9,100,123,CD,0,0,0,0,0,0,0,...,6,33,1,0,2,50,3,67,3,10


In [25]:
mikala_CD = mikala_numeric.loc[mikala_numeric['Position'] == 'CD']
mikala_DM = mikala_numeric.loc[mikala_numeric['Position'] == 'DM']
mikala_RD = mikala_numeric.loc[mikala_numeric['Position'] == 'RD']


In [26]:
# mikala_numeric.corr(method='pearson').round(2)

In [27]:
mikcorr = mikala_numeric.corr(method='spearman').round(2)
# mikcorr.to_excel('Mikala_Correlation.xlsx', engine = 'xlsxwriter')

In [28]:
rounded_CD = mikala_CD.corr(method='spearman').round(2)

# Print Meaningful Correlations in Top 2 Positions

In [29]:
def important(Dataframe):
    Dataframe_important = Dataframe.iloc[:,0:2]
    Dataframe_important = Dataframe_important.loc[(Dataframe_important['Score'].abs() > 0.4) | (Dataframe_important['InStat Index'].abs() > 0.4)]
    return Dataframe_important

important(rounded_CD)

,Score,InStat Index
Score,1.00,0.06
InStat Index,0.06,1.00
Fouls,-0.41,-0.24
Yellow cards,-0.65,0.17
Shots,-0.33,0.57
Shots on target,-0.65,0.17
xG (Expected goals),-0.65,0.17
Crosses,0.33,0.76
Lost balls,-0.34,-0.43
Lost balls in own half,-0.69,-0.19


In [30]:
rounded_DM = mikala_DM.corr(method='spearman').round(2)
important(rounded_DM)

,Score,InStat Index
Score,1.00,0.00
InStat Index,0.00,1.00
Assists,0.50,0.87
Сhances created,0.50,0.87
Fouls,0.50,0.87
Passes,0.87,0.50
Key passes,0.50,0.87
Crosses,0.50,0.87
Lost balls,0.87,-0.50
Lost balls in own half,0.87,-0.50
